Data wrangling the Stats's Json and generate stats.csv file

In [1]:
from pathlib import Path
import os
import pandas as pd
import json

In [3]:
# Define paths for JSON input and CSV output
json_folder_path = Path.cwd().parents[1] / 'json_renamed'
csv_folder_path = Path.cwd().parents[1] / 'csv_datasets'

# Ensure the CSV directory exists
csv_folder_path.mkdir(parents=True, exist_ok=True)

In [4]:
def flatten_json(data, parent_key='', sep='_'):
    """
    Recursively flattens a nested JSON dictionary.
    """
    items = {}
    for k, v in data.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_json(v, new_key, sep=sep))
        elif isinstance(v, list):
            for i, item in enumerate(v):
                if isinstance(item, dict):
                    items.update(flatten_json(item, f"{new_key}{i}", sep=sep))
                else:
                    items[f"{new_key}{i}"] = item
        else:
            items[new_key] = v
    return items

def extract_stats(data):
    """
    Extracts and flattens all statistics data into a standardized format.
    """
    flattened_data = []

    # Loop through each period in the JSON data
    for period in data.get('statistics', []):
        period_flat = flatten_json(period, 'period')
        # Extract statistics for each group
        for group in period.get('groups', []):
            group_flat = flatten_json(group, 'group')
            # Extract statistics for each item in the group
            for stat_item in group.get('statisticsItems', []):
                item_flat = flatten_json(stat_item, 'item')
                # Combine period, group, and item data
                combined_flat = {**period_flat, **group_flat, **item_flat}
                flattened_data.append(combined_flat)

    # Return a DataFrame from the flattened data
    return pd.DataFrame(flattened_data)

# List to store DataFrames for each file
stats_dataframes = []

# Iterate over all files in the JSON folder
for filename in os.listdir(json_folder_path):
    try:
        # Process only JSON files that contain 'stats' in their name
        if filename.endswith('.json') and 'stats' in filename:
            json_file_path = os.path.join(json_folder_path, filename)

            with open(json_file_path, 'r', encoding='utf-8') as file:
                json_data = json.load(file)

            # Check if 'statistics' data exists in the JSON
            if 'statistics' in json_data:
                # Extract statistics data from the JSON file
                df = extract_stats(json_data)

                # Add 'date' and 'code' from the filename to the DataFrame
                date, code, _ = filename.split('_')[1:4]
                df.insert(0, 'date', date)
                df.insert(1, 'code', code)

                # Append the DataFrame to the list
                stats_dataframes.append(df)

    except Exception as e:
        print(f"Error processing file {filename}: {str(e)}")


Consolidated data saved to /Users/fernandaalves/Documents/code_studies/palmeiras_analytics_br/csv_datasets/stats.csv


In [ ]:
# Filter out empty or all-NA DataFrames
stats_dataframes = [df for df in stats_dataframes if not df.empty and not df.isna().all().all()]

# Concatenate all valid DataFrames into a single DataFrame
if stats_dataframes:
    result_df = pd.concat(stats_dataframes, ignore_index=True)
    # Save the consolidated DataFrame to a CSV file
    result_df.to_csv(csv_folder_path / 'stats.csv', index=False)
    print(f"Consolidated data saved to {csv_folder_path / 'stats.csv'}")
else:
    print('No DataFrame to concatenate.')

Key Improvements

    Dynamic Data Extraction: The updated flatten_json function uses a recursive approach to extract all nested information, ensuring that no data is missed regardless of the depth of the structure.
    Flexible Data Handling: The code dynamically adapts to the different JSON structures, ensuring that all data is extracted even if the format varies between files.
    Efficient Data Concatenation: Only valid DataFrames (non-empty and containing at least some non-NA values) are concatenated, minimizing the risk of introducing empty or invalid rows into the final CSV.